# Carga de librerías y directorios

In [1]:
import pandas as pd
import numpy as np

# Métrica de evaluación
from sklearn.metrics import f1_score
# fold_score = f1_score(y_test, prediction, average='micro')
# Se especifica average por tener un target multiclase

# Clasificador LightGBM
from lightgbm import LGBMClassifier

# Para la división en k-folds
from sklearn.model_selection import KFold
# Para ver "progreso" inline: limpia la celda en cada iteración
from IPython.display import clear_output

# Librería para guardar los Trials.
import pickle

# Función para cambiar tipos de datos
from resources.utilidades import cambio_tipos

# Rutas de los archivos a usar

In [2]:
# Ruta train_values.csv
dir_values = 'data/train_values.csv'
# Ruta train_labels.csv
dir_labels = 'data/train_labels.csv'
# Ruta test_values.csv
dir_test = 'data/test_values.csv'
# Ruta submission_format.csv
dir_sub_format = 'data/submission_format.csv'
# Ruta de Binary Encodings para train_values.csv
dir_resources = 'resources'

# Entrenamiento

### Train Set

In [3]:
columnas = ['building_id',
 'geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'count_floors_pre_eq',
 'age',
 'area_percentage',
 'height_percentage',
 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_superstructure_other',
 'count_families',
 'has_secondary_use',
 'has_secondary_use_agriculture',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'has_secondary_use_institution',
 'has_secondary_use_school',
 'has_secondary_use_industry',
 'has_secondary_use_health_post',
 'has_secondary_use_gov_office',
 'has_secondary_use_use_police',
 'has_secondary_use_other']

# Carga de train_values.csv
train = pd.read_csv(dir_values, usecols=columnas)

### Cambio de tipos de datos

In [4]:
# Se cambian los tipos de datos para reducir uso de memoria
train = cambio_tipos(train)

### Carga de columnas codificadas: One Hot Encoding

In [5]:
# 38 columnas más
train = train.join(pd.read_csv(dir_resources+f'/OHE_train.csv', dtype='uint8'))

In [6]:
# Se le quitan columnas
to_drop = ['building_id', 'has_secondary_use_school', 'has_secondary_use_health_post', \
           'has_secondary_use_gov_office',  'has_secondary_use_use_police', \
           'C_a', 'C_f', 'C_m', 'C_n', 'C_o']
train = train.drop(columns=to_drop)

# GridSearchCV, RandomizedSerach, Bayesian Search y LightGBMClassifier necesitan, o funcionan mejor con, los valores en una dimensión.
# Valores del data set
X = train.values

### Carga de labels

In [7]:
# Carga de train_labels.csv
labels = pd.read_csv(dir_labels, usecols=['damage_grade'], dtype='uint8')

# GridSearchCV, RandomizedSerach, Bayesian Search y LightGBMClassifier necesitan, o funcionan mejor con, los valores en una dimensión.
# Valores del data set
y = labels.damage_grade.values

### Test Set

In [8]:
# Carga de test_values.csv
test = pd.read_csv(dir_test, usecols=columnas)

### Cambio de tipos de datos

In [9]:
test = cambio_tipos(test)

### Carga de columnas codificadas: One Hot Encoding

In [10]:
# 28 columnas más
test = test.join(pd.read_csv(dir_resources+f'/OHE_test.csv', dtype='uint8'))

# Se le quitan columnas
to_drop = ['building_id', 'has_secondary_use_school', 'has_secondary_use_health_post', \
           'has_secondary_use_gov_office',  'has_secondary_use_use_police', \
           'C_a', 'C_f', 'C_m', 'C_n', 'C_o']
test = test.drop(columns=to_drop)

# Valores del data set
test = test.values

### Instancia con parámetros optimizados

In [11]:
params = {
    'learning_rate' : 0.0691427312603393,
    'num_leaves' : 112,
    'max_depth' : 61,
    'n_estimators' : 880,
    'importance_type' : 'split'
}

lgbm = LGBMClassifier()
# Se setean los parámetros desde el diccionario
lgbm.set_params(**params)
lgbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.0691427312603393,
 'max_depth': 61,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 880,
 'n_jobs': -1,
 'num_leaves': 112,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

```
# n_splits
splits = 9

# Dataframe con predicciones
predictions_train = pd.DataFrame()
predictions_test = pd.DataFrame()
```

### Carga de Dataframe con predicciones anteriores

In [15]:
predictions_train = pd.read_csv('pred_train_dataframe.csv')
predictions_test = pd.read_csv('pred_test_dataframe.csv')

In [17]:
print(predictions_train.shape)
print(predictions_test.shape)

(260601, 45)
(86868, 45)


### Acá se entrenan los modelos con los hiper parámetros optimizados

In [24]:
## Creación de DataFrames que guardan las predicciones.
# predictions_train = pd.DataFrame(index=train.index)
## Recordar cargar el test para tener el índice.
# predictions_test = pd.DataFrame(index=test.index)

In [18]:
%%time
##########
# SPLITS #

splits=5

#########

kf = KFold(n_splits=splits)

# Listas para guardar los puntajes y errores de la evaluación
train_score = []
test_score = []
i = 1

# K-Validations
for train_index, test_index in kf.split(X, y) :
    clear_output(wait=True)

    # Se dividen los datos con los índice del split de Kfold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print('Entrenando fold '+str(i)+'/'+str(splits))
    # Entrenamiento del k-fold y predicción
    lgbm.fit(X_train, y_train)
    prediction_train = lgbm.predict(X_train)
    prediction_test = lgbm.predict(X_test)

    # Se guardan los modelos que intervienen
    file_name = 'models/LighGBM_Classifier_'+str(splits)+'KFold_'+str(i)+'.pkl'
    model_pickle = open(file_name, 'wb')
    pickle.dump(lgbm, model_pickle)
    model_pickle.close()

    # Dataframes: agregado de columnas de predicciones para train evaluation y submission
    predictions_train[str(splits)+'KFold_'+str(i)] = lgbm.predict(train)
    predictions_test[str(splits)+'KFold_'+str(i)] = lgbm.predict(test)

    # Se agregan los scores a las listas
    train_score.append(f1_score(y_train, prediction_train, average='micro'))
    test_score.append(f1_score(y_test, prediction_test, average='micro'))

    i = i + 1

Entrenando fold 5/5
Wall time: 3min 43s


In [19]:
print(predictions_train.shape)
print(predictions_test.shape)

(260601, 45)
(86868, 45)


In [20]:
print('Train validation mean:', np.mean(train_score).round(4), \
      'std:', np.std(train_score).round(4))
print('Test validation  mean:', np.mean(test_score).round(4), \
      'std:', np.std(test_score).round(4))

Train validation mean: 0.8407 std: 0.0007
Test validation  mean: 0.7454 std: 0.0016


### Test Validation
```
 5Fold mean: 0.7454 std: 0.0016
 6Fold mean: 0.7458 std: 0.0014
 7Fold mean: 0.7460 std: 0.0015
 8Fold mean: 0.7459 std: 0.0023
 9Fold mean: 0.7462 std: 0.0027
10Fold mean: 0.7463 std: 0.0032
```
### Train Validation
```
 5Fold mean: 0.8407 std: 0.0007
 6Fold mean: 0.8414 std: 0.0005
 7Fold mean: 0.8441 std: 0.0011
 8Fold mean: 0.8295 std: 0.0006
 9Fold mean: 0.8333 std: 0.0007
10Fold mean: 0.8743 std: 0.0005
```

Ahora busco modas en el DataFrame

In [37]:
%%time
moda_train = predictions_train.mode(axis=1)[0]

Wall time: 1min 18s


In [38]:
# Distribución de predicciones
moda_train.value_counts()

2.0    164533
3.0     74841
1.0     21227
Name: 0, dtype: int64

### Distribuciones de resultados para distintos K-Folds
```
5Fold
2.0    164533
3.0     74841
1.0     21227

6Fold
2.0    164565
3.0     74640
1.0     21396

7Fold
2.0    164119
3.0     75153
1.0     21329

8Fold
2.0    164119
3.0     75153
1.0     21329

9Fold
2.0    162850
3.0     75964
1.0     21787

10fold
2.0    161359
3.0     76312
1.0     22930
```

In [39]:
# Score
f1_score(y, moda_train, average='micro')

0.8414741309511476

### F1 Score: Train Validation
```
 5Fold: 0.8414741309511476
 6Fold: 0.842379729931965
 7Fold: 0.8446398901001915
 8Fold: 0.8446398901001915
 9Fold: 0.8540796082900679
10Fold: 0.8723297301238292
```

# Persistencia

In [40]:
predictions_train.to_csv('pred_train_dataframe.csv', index=False)
predictions_test.to_csv('pred_test_dataframe.csv', index=False)

In [41]:
predictions_train.columns

Index(['10KFold_1', '10KFold_2', '10KFold_3', '10KFold_4', '10KFold_5',
       '10KFold_6', '10KFold_7', '10KFold_8', '10KFold_9', '10KFold_10',
       '9KFold_1', '9KFold_2', '9KFold_3', '9KFold_4', '9KFold_5', '9KFold_6',
       '9KFold_7', '9KFold_8', '9KFold_9', '8KFold_1', '8KFold_2', '8KFold_3',
       '8KFold_4', '8KFold_5', '8KFold_6', '8KFold_7', '8KFold_8', '7KFold_1',
       '7KFold_2', '7KFold_3', '7KFold_4', '7KFold_5', '7KFold_6', '7KFold_7',
       '6KFold_1', '6KFold_2', '6KFold_3', '6KFold_4', '6KFold_5', '6KFold_6',
       '5KFold_1', '5KFold_2', '5KFold_3', '5KFold_4', '5KFold_5'],
      dtype='object')

# Para predicción del Test
Se pueden elegir las columnas que tiene las distintas predicciones de los modelos entrenados.

In [29]:
%%time
moda_test = predictions_test.mode(axis=1)[0]

Wall time: 25.4 s


In [7]:
sub_example = pd.read_csv(dir_sub_format)

sub_example['damage_grade'] = moda_test.astype('uint8')

In [8]:
# Nombre de submission
file_name = 'submission11.csv'

In [9]:
sub_example.to_csv(file_name, index=False)

In [10]:
sub_example

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3
...,...,...
86863,310028,2
86864,663567,2
86865,1049160,2
86866,442785,2


In [11]:
# Verificación de archivo para submit
aux = pd.read_csv(file_name)

aux

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3
...,...,...
86863,310028,2
86864,663567,2
86865,1049160,2
86866,442785,2


# Pickle

In [23]:
# String con dirección de almacenamiento y nombre del archivo .pkl.
lgbm_file_name = 'LightGBM_Classifier07.pkl'

# Se crea una variable pickle y se abre en modo de escritura.
# [También se crea el archivo en el directorio especificado por lgbm_file_name]
model_pickle = open(lgbm_file_name, 'wb')

# Se usa el método dump con el modelo y el archivo creado en modo de esritura
# para crear el pickle.
pickle.dump(lgbm, model_pickle)

# Finalmente hay que cerrar la instancia del pickle.
model_pickle.close()

# Lectura de Pickle

In [ ]:
# String con dirección de almacenamiento y nombre del archivo .pkl.
file_name = 'LightGBM_Classifier06.pkl'

# Se crea una variable para el archivo pickle y se abre con la dirección del
# archivo en modo lectura. 
model_predictor_pkl = open(file_name, 'rb')

# Se carga el modelo pasando la variable creada, en la misma se encuentra
# la dirección del archivo pickle.
model_predictor = pickle.load(model_predictor_pkl)
print('Modelo cargado')
model_predictor.get_params